In [ ]:
## Merge img_meta into the dataset json file 
## name+description under the key "caption", page under the key "url", src under the key "imgUrl", 
## image id under the key "image_id"

### Next step: see "select_hard_distractors" for ranking distractos by RE w.r.t Q 
### (with a bit length penalty/bonus to balance length bias)

In [1]:
import json, random, time
import numpy as np
from pprint import pprint
from collections import Counter, defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import os
import re
from nltk.corpus import stopwords
import pickle
import pylcs
np.set_printoptions(precision=4)

In [2]:
txt_dataset_remove_clean_longimg_tmp_0817 = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_remove_clean_longimg_tmp_0817.json", "r"))

In [11]:
txt_dataset_remove_clean_longimg_tmp_0817['10']['img_negFacts'][0]

{'caption': 'Oscar Zariski. Oscar Zariski (1899–1986)',
 'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/e/e5/Oscar_Zariski.jpg',
 'scores': '(0.05, 0.0, 0.0, 0.05, 0.05)',
 'title': 'Oscar Zariski',
 'url': 'https://en.wikipedia.org/wiki/Oscar_Zariski'}

In [6]:
img_meta = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data/img_metadata-Copy1.json", "r"))

In [12]:
img_meta['0'].keys()

dict_keys(['src', 'page', 'name', 'description'])

In [10]:
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0812.json", "r"))

In [27]:
img_dataset_merge_meta_tmp_0817 = {}
count = 0
for k in img_dataset:
    img_dataset_merge_meta_tmp_0817[k] = copy.deepcopy(img_dataset[k])
    img_dataset_merge_meta_tmp_0817[k]['img_posFacts'] = []
    img_dataset_merge_meta_tmp_0817[k]['img_negFacts'] = []
    for g in img_dataset[k]['GoldIds']:
        img_meta_key = str(int(g))
        m = img_meta[img_meta_key]
        img_dataset_merge_meta_tmp_0817[k]['img_posFacts'].append({
            'image_id': g,
            'title': m['name'].replace("_", " ").replace(".JPG", " ").strip(),
            'caption': (m['name'] + m['description']).replace("_", " ").replace(".JPG", " ").strip(),
            'url': m['page'],
            'imgUrl': m['src']
        })
    for g in img_dataset[k]['DistractorIds']:
        img_meta_key = str(int(g))
        m = img_meta[img_meta_key]
        img_dataset_merge_meta_tmp_0817[k]['img_negFacts'].append({
            'image_id': g,
            'title': m['name'].replace("_", " ").replace(".JPG", " ").strip(),
            'caption': (m['name'] + m['description']).replace("_", " ").replace(".JPG", " ").strip(),
            'url': m['page'],
            'imgUrl': m['src']
        })
    del img_dataset_merge_meta_tmp_0817[k]['GoldIds']
    del img_dataset_merge_meta_tmp_0817[k]['DistractorIds']
    count += 1
    if count%5000 == 0: print(count)

print(len(img_dataset_merge_meta_tmp_0817))
json.dump(img_dataset_merge_meta_tmp_0817, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_merge_meta_tmp_0817.json", "w"), indent=4)


5000
10000
15000
20000
25000
25452


In [28]:
pprint(img_dataset_merge_meta_tmp_0817['3408'])

{'A': '"The wrestler with the single kneepad wore cornrows."',
 'DistractorFacts': [{'fact': 'In the qualifying tournament for the 2016 Rio '
                              'Olympics held in Istanbul, she won in the final '
                              'round beating Polish wrestler Iwona Matkowska, '
                              'and qualified for the Olympics in the process.',
                      'scores': '(0.13, 0.0, 0.13)',
                      'title': 'Vinesh Phogat',
                      'url': 'https://en.wikipedia.org/wiki/Vinesh_Phogat'},
                     {'fact': 'Athletes from 87 nations earned medals at the '
                              '2016 Summer Olympics, breaking the record of '
                              'most nations winning a medal at a single '
                              'edition of the Olympics.',
                      'scores': '(0.1, 0.0, 0.1)',
                      'title': '2016 Summer Olympics medal table',
                      'url': 'ht

### Merge img_meta 0901 after re-mining descriptions from wikimedia-commons

In [28]:
img_meta = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_meta_0901_shorter_truncate.json", "r"))
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0823_clean_te.json", "r"))

print(Counter([img_dataset[k]['split'] for k in img_dataset]))
print(Counter([img_dataset[k]['Qcate'] for k in img_dataset]))
print(Counter([img_dataset[k]['Qcate'] for k in img_dataset if img_dataset[k]['split'] == 'test']))
print(len(set([img_dataset[k]['Guid'] for k in img_dataset])))

Counter({'train': 16448, 'ood_test': 3948, 'val': 2511, 'ind_test': 2485})
Counter({'YesNo': 8410, 'Others': 6689, 'choose': 5226, 'number': 2337, 'color': 2068, 'shape': 662})
Counter()
25392


In [26]:
img_dataset_0901_remine_des = {}
count = 0
for k in img_dataset:
    img_dataset_0901_remine_des[k] = copy.deepcopy(img_dataset[k])
    img_dataset_0901_remine_des[k]['img_posFacts'] = []
    img_dataset_0901_remine_des[k]['img_negFacts'] = []
    for f in img_dataset[k]['img_posFacts']:
        m = img_meta[f['image_id']]
        new_f = copy.deepcopy(f)
        new_f['caption'] = " ".join((m['name'] + " " + m['description']).split()).replace("_", " ").replace(".JPG", " ").strip()
        img_dataset_0901_remine_des[k]['img_posFacts'].append(new_f)
    for f in img_dataset[k]['img_negFacts']:
        m = img_meta[f['image_id']]
        new_f = copy.deepcopy(f)
        new_f['caption'] = " ".join((m['name'] + " " + m['description']).split()).replace("_", " ").replace(".JPG", " ").strip()
        img_dataset_0901_remine_des[k]['img_negFacts'].append(new_f)
    count += 1
    if count%5000 == 0: print(count)

print(len(img_dataset_0901_remine_des))
json.dump(img_dataset_0901_remine_des, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0901_remine_des_shorter_truncate.json", "w"), indent=4)


5000
10000
15000
20000
22423


In [25]:
k = random.choice(list(img_dataset_0901_remine_des.keys()))
pprint(img_dataset_0901_remine_des[k])

{'A': ['"There are books shelved behind the woman."'],
 'Guid': 'dc5c0554d46f11ebba07a504a20e7724',
 'Keywords_A': '"Books."',
 'Q': '"What is shelved behind the woman depicted in Nicolas de LARGILLIÈRe\'s '
      'Frances Woollascott, an Augustinian Nun?"',
 'Qcate': 'Others',
 'Qtype': 'img-Singlehop',
 'img_negFacts': [{'caption': 'Nicolas de LARGILLIÈRe - Frances Woollascott, '
                              'an Augustinian Nun - Google Art Project Gift of '
                              'William Bowmore AO OBE through the Art Gallery '
                              'of South Australia Foundation 1991',
                   'image_id': '00012085',
                   'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2e/Nicolas_de_LARGILLI%C3%88Re_-_Frances_Woollascott%2C_an_Augustinian_Nun_-_Google_Art_Project.jpg/464px-Nicolas_de_LARGILLI%C3%88Re_-_Frances_Woollascott%2C_an_Augustinian_Nun_-_Google_Art_Project.jpg',
                   'title': 'Nicolas de LARGILLIÈRe -

In [19]:
for k in random.sample(list(img_meta.keys()), 30):
    pprint(img_meta[k])

{'description': 'The Santa Clara Vanguard, 7/26/08.',
 'name': 'Santaclaravanguard08',
 'page': 'https://commons.wikimedia.org/wiki/File:Santaclaravanguard08.jpg',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ee/Santaclaravanguard08.jpg/1280px-Santaclaravanguard08.jpg'}
{'description': '',
 'name': 'USS_South_Dakota_in_Ulithi_NARA_BuAer_294134',
 'page': 'https://commons.wikimedia.org/wiki/File:USS_South_Dakota_in_Ulithi_NARA_BuAer_294134.jpg',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/USS_South_Dakota_in_Ulithi_NARA_BuAer_294134.jpg/744px-USS_South_Dakota_in_Ulithi_NARA_BuAer_294134.jpg'}
{'description': '',
 'name': 'Cabinet_du_Conseil._Versailles._05.JPG',
 'page': 'https://commons.wikimedia.org/wiki/File:Cabinet_du_Conseil._Versailles._05.JPG',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/9/96/Cabinet_du_Conseil._Versailles._05.JPG/1200px-Cabinet_du_Conseil._Versailles._05.JPG'}
{'description': 'Sardinian Woman wearing th